# Support Vector Machine (SVM)

## By: Mustafa Yildirim

Implementing binary classification using Support Vector Machines (SVMs) to distinguish between digits '7' and '9'. 

Training both a linear SVM and a nonlinear SVM with a Gaussian RBF kernel, while fine-tuning hyperparameters (C and γ) for optimal performance. 

Evaluating the classification accuracy on held-out test data and comparing the Scikit-learn implementation with the Projected Gradient Descent (PGD) optimizer in terms of accuracy and computational efficiency.

In [ ]:
!pip install -U -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# load MNIST data using Datasets at Hugging Face
#
from datasets import load_dataset
import numpy as np

trainset = load_dataset('mnist', split='train')
train_data = trainset['image']
train_label = trainset['label']

testset = load_dataset('mnist', split='test')
test_data = testset['image']
test_label = testset['label']

train_data = np.array(train_data, dtype='float')/255 # norm to [0,1]
train_data = np.reshape(train_data,(60000,28*28))
train_label = np.array(train_label, dtype='short')
test_data = np.array(test_data, dtype='float')/255 # norm to [0,1]
test_data = np.reshape(test_data,(10000,28*28))
test_label = np.array(test_label, dtype='short')

print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

(60000, 784) (60000,) (10000, 784) (10000,)


In [ ]:
# prepare digits '7' and '9' for binary SVMs
digit_train_index = np.logical_or(train_label == 7, train_label == 9)
X_train = train_data[digit_train_index]
y_train = train_label[digit_train_index]
digit_test_index = np.logical_or(test_label == 7, test_label == 9)
X_test = test_data[digit_test_index]
y_test = test_label[digit_test_index]

# normalize all feature vectors to unit-length
X_train = np.transpose (X_train.T / np.sqrt(np.sum(X_train*X_train, axis=1)))
X_test =  np.transpose (X_test.T  / np.sqrt(np.sum(X_test*X_test, axis=1)))

# convert labels: '7' => -1, '9' => +1
CUTOFF = 8 # any number between '7' and '9'
y_train = np.sign(y_train-CUTOFF)
y_test = np.sign(y_test-CUTOFF)

Sklearn Linear SVM

Displaying the accuracy with different C values

In [ ]:
# linear SVM: use sk-learn SVC functions
import numpy as np
from sklearn.svm import SVC

for c in [0.0001, 0.001, 0.01, 0.1, 1, 2, 4, 10, 20, 50, 100, 500, 1000]:
  linearSVM = SVC(kernel='linear', C=c)
  linearSVM.fit(X_train,y_train)
  predict = linearSVM.predict(X_train)
  train_acc = np.count_nonzero(np.equal(predict,y_train))/y_train.size
  predict = linearSVM.predict(X_test)
  test_acc = np.count_nonzero(np.equal(predict,y_test))/y_test.size
  print(f'linear SVM (C={c}): training accuracy={100*train_acc:.2f}%  test accuracy={100*test_acc:.2f}%')


linear SVM (C=0.0001): training accuracy=51.29%  test accuracy=50.47%
linear SVM (C=0.001): training accuracy=74.64%  test accuracy=77.86%
linear SVM (C=0.01): training accuracy=93.95%  test accuracy=94.65%
linear SVM (C=0.1): training accuracy=95.60%  test accuracy=96.12%
linear SVM (C=1): training accuracy=96.39%  test accuracy=96.51%
linear SVM (C=2): training accuracy=96.72%  test accuracy=96.71%
linear SVM (C=4): training accuracy=96.95%  test accuracy=96.86%
linear SVM (C=10): training accuracy=97.05%  test accuracy=97.01%
linear SVM (C=20): training accuracy=97.23%  test accuracy=97.01%
linear SVM (C=50): training accuracy=97.45%  test accuracy=96.86%
linear SVM (C=100): training accuracy=97.54%  test accuracy=96.76%
linear SVM (C=500): training accuracy=97.74%  test accuracy=96.51%
linear SVM (C=1000): training accuracy=97.85%  test accuracy=96.42%


Sklearn Nonlinear SVM with Gaussian RBF kernel

Displaying the accuracy with different C and gamma values

In [ ]:
# nonlinear SVM (Gaussian RBF kernel): use sk-learn SVC functions
for c in [0.01, 0.1, 1, 10, 100, 1000]:
  for g in ['scale', 0.001, 0.01, 0.1, 1, 10, 100]:
    rbfSVM = SVC(kernel='rbf', C=c, gamma=g)
    rbfSVM.fit(X_train,y_train)
    predict = rbfSVM.predict(X_train)
    train_acc = np.count_nonzero(np.equal(predict,y_train))/y_train.size
    predict = rbfSVM.predict(X_test)
    test_acc = np.count_nonzero(np.equal(predict,y_test))/y_test.size
    print(f'nonlinear RBF SVM (C={c},gamma={g}): training accuracy={100*train_acc:.2f}%  test accuracy={100*test_acc:.2f}%')

nonlinear RBF SVM (C=0.01,gamma=scale): training accuracy=95.48%  test accuracy=95.78%
nonlinear RBF SVM (C=0.01,gamma=0.001): training accuracy=51.29%  test accuracy=50.47%
nonlinear RBF SVM (C=0.01,gamma=0.01): training accuracy=51.29%  test accuracy=50.47%
nonlinear RBF SVM (C=0.01,gamma=0.1): training accuracy=92.41%  test accuracy=93.08%
nonlinear RBF SVM (C=0.01,gamma=1): training accuracy=95.31%  test accuracy=95.63%
nonlinear RBF SVM (C=0.01,gamma=10): training accuracy=51.29%  test accuracy=50.47%
nonlinear RBF SVM (C=0.01,gamma=100): training accuracy=51.29%  test accuracy=50.47%
nonlinear RBF SVM (C=0.1,gamma=scale): training accuracy=97.87%  test accuracy=97.74%
nonlinear RBF SVM (C=0.1,gamma=0.001): training accuracy=51.29%  test accuracy=50.47%
nonlinear RBF SVM (C=0.1,gamma=0.01): training accuracy=92.41%  test accuracy=93.27%
nonlinear RBF SVM (C=0.1,gamma=0.1): training accuracy=94.63%  test accuracy=95.19%
nonlinear RBF SVM (C=0.1,gamma=1): training accuracy=97.67%  t

PGD Linear SVM

In [ ]:
# solve linear SVMs using projected gradient descent (PGD)

class mySVM1():
  def __init__(self, kernel='linear', optimizer='pgd', debug=0, threshold=0.001, \
               lr=1.0, max_epochs=10, batch_size=2, C=1):
    self.kernel = kernel        # kernel type
    self.optimizer = optimizer  # which optimizer is used to solve quadratic programming
    self.lr = lr                # max learning rate in PGD
    self.max_epochs = max_epochs   # max epochs in PGD
    self.batch_size = batch_size   # size of each subset in PGD
    self.debug = debug             # whether print debugging info
    self.threshold = threshold     # threshold to filter out support vectors

    self.C = C     # C for the soft-margin term

  # Linear Kernel Function
  # X[N,d]: training samples;  Y[M,d]: other training samples
  # return Q[N,N]: linear kernel matrix between X and Y
  def Kernel(self, X, Y):
    if (self.kernel == 'linear'):
      K = X @ Y.T

    return K

  # construct matrix Q from any kernel function for dual SVM optimization
  def QuadraticMatrix(self, X, y):
    Q = np.outer(y, y) * self.Kernel(X, X)
    return Q

  # use projected gradient descent to solve quadratic program
  # refer to Algorithm 6.5 on page 127
  # Q[N,N]: quadratic matrix;  y[N]: training labels (+1 or -1)
  def PGD(self, Q, y):
    N = Q.shape[0]   # num of training samples
    alpha = np.zeros(N)
    prev_L = 0.0

    for epoch in range(self.max_epochs):
      indices = np.random.permutation(N)  #randomly shuffle data indices
      for batch_start in range(0, N, self.batch_size):
        idx = indices[batch_start:batch_start + self.batch_size] # indices of the selected subset
        alpha_s = alpha[idx]
        y_s = y[idx]

        grad_s = Q[idx,:] @ alpha - np.ones(idx.shape[0])
        proj_grad_s = grad_s - np.dot(y_s,grad_s)/np.dot(y_s, y_s)*y_s

        bound = np.zeros(idx.shape[0])
        bound[proj_grad_s < 0] = self.C

        eta = np.min(np.abs(alpha_s-bound)/(np.abs(proj_grad_s)+0.001))

        alpha[idx] -= min(eta, self.lr) * proj_grad_s

      L = 0.5 * alpha.T @ Q @ alpha - np.sum(alpha) # objectibve function
      if (L > prev_L):
        if (self.debug>0):
          print('Early stopping at epoch={epoch}!')
        break

      if (self.debug>1):
        print(f'[PGD optimizer] epoch = {epoch}: L = {L:.5f}  (# of support vectors = {(alpha>self.threshold).sum()})')
        print(f'                 alpha: max={np.max(alpha)} min={np.min(alpha)} orthogonal constraint={np.dot(alpha,y):.2f}')

      prev_L = L

    return alpha

  # train SVM from training samples
  # X[N,d]: input features;  y[N]: output labels (+1 or -1)
  def fit(self, X, y):
    if(self.kernel != 'linear'):
      print("Error: only linear kernel is supported!")
      return

    Q = self.QuadraticMatrix(X, y)

    alpha = self.PGD(Q, y)

    #save support vectors (pruning all data with alpha==0)
    self.X_SVs = X[alpha>self.threshold]
    self.y_SVs = y[alpha>self.threshold]
    self.alpha_SVs = alpha[alpha>self.threshold]

    # compute weight vector for linear SVMs (refer to the formula on page 120)
    if(self.kernel == 'linear'):
      self.w = (self.y_SVs * self.alpha_SVs) @ self.X_SVs

    # estimate b
    idx = np.nonzero(np.logical_and(self.alpha_SVs>self.threshold,self.alpha_SVs<self.C-self.threshold))
    if(len(idx) == 0):
      idx = np.nonzero(self.alpha_SVs>self.threshold)
    # refer to the formula on page 125 (above Figure 6.11)
    b = self.y_SVs[idx] - (self.y_SVs * self.alpha_SVs) @ self.Kernel(self.X_SVs, self.X_SVs[idx])
    self.b = np.median(b)

    return

  # use SVM from prediction
  # X[N,d]: input features
  def predict(self, X):
    if(self.kernel != 'linear'):
      print("Error: only linear kernel is supported!")
      return

    y = X @ self.w + self.b

    return np.sign(y)


Displaying the accuracy with different C values

In [ ]:
for c in [0.1, 1, 2, 4, 10]:
  svm = mySVM1(max_epochs=10, lr=2.0, C=c, kernel='linear')
  svm.fit(X_train,y_train)

  predict = svm.predict(X_train)
  train_acc = np.count_nonzero(np.equal(predict,y_train))/y_train.size
  predict = svm.predict(X_test)
  test_acc = np.count_nonzero(np.equal(predict,y_test))/y_test.size
  print(f'MY linear SVM (C={c}): training accuracy={100*train_acc:.2f}%  test accuracy={100*test_acc:.2f}%')

MY linear SVM (C=0.1): training accuracy=95.40%  test accuracy=96.17%
MY linear SVM (C=1): training accuracy=95.85%  test accuracy=96.27%
MY linear SVM (C=2): training accuracy=96.32%  test accuracy=96.71%
MY linear SVM (C=4): training accuracy=96.26%  test accuracy=96.17%
MY linear SVM (C=10): training accuracy=95.97%  test accuracy=96.22%


Displaying the timing

In [ ]:
c=1

# Scikit-learn Linear SVM
linearSVM = SVC(kernel='linear', C=c)
%timeit linearSVM.fit(X_train,y_train)

# PGD-based linear SVM
svm = mySVM1(max_epochs=10, lr=2.0, C=c, kernel='linear')
%timeit svm.fit(X_train,y_train)

15.4 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
11.6 s ± 407 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


PGD Nonlinear SVM with Gaussian RBF kernel

In [ ]:
# extend for nonlinear SVMs by adding polynomial and RBF kernel functions
#

class mySVM2():
  def __init__(self, kernel='linear', optimizer='pgd', debug=0, threshold=0.001, \
               lr=1.0, max_epochs=20, batch_size=2, C=1, order=3, gamma=1.0):
    self.kernel = kernel           # kernel type
    self.optimizer = optimizer     # which optimizer is used to solve quadratic programming
    self.lr = lr                   # max learning rate in PGD
    self.max_epochs = max_epochs   # max epochs in PGD
    self.batch_size = batch_size   # size of each subset in PGD
    self.debug = debug             # whether print debugging info
    self.threshold = threshold     # threshold to filter out support vectors

    self.C = C                     # C for the soft-margin term
    self.order = order             # power order for polynomial kernel
    self.gamma = gamma             # gamma for Gaussian RBF kernel

  # Kernel Function
  # X[N,d]: training samples;  Y[M,d]: other training samples
  # return Q[N,N]: linear kernel matrix between X and Y
  def Kernel(self, X, Y):
    if (self.kernel == 'linear'):
      K = X @ Y.T
    elif (self.kernel == 'poly'):
      K = np.power(X @ Y.T +1, self.order)
    elif (self.kernel == 'rbf'):
      d1 = np.sum(X*X, axis=1)
      d2 = np.sum(Y*Y, axis=1)
      K = np.outer(d1, np.ones(Y.shape[0])) + np.outer(np.ones(X.shape[0]), d2) \
          - 2 * X @ Y.T
      K = np.exp(-self.gamma * K)

    return K

  # construct matrix Q from any kernel function for dual SVM optimization
  def QuadraticMatrix(self, X, y):
    Q = np.outer(y, y) * self.Kernel(X, X)
    return Q

  # use projected gradient descent to solve quadratic program
  # refer to Algorithm 6.5 on page 127
  # Q[N,N]: quadratic matrix;  y[N]: training labels (+1 or -1)
  def PGD(self, Q, y):
    N = Q.shape[0]   # num of training samples
    alpha = np.zeros(N)
    prev_L = 0.0

    for epoch in range(self.max_epochs):
      indices = np.random.permutation(N)  #randomly shuffle data indices
      for batch_start in range(0, N, self.batch_size):
        idx = indices[batch_start:batch_start + self.batch_size] # indices of the current subset
        alpha_s = alpha[idx]
        y_s = y[idx]

        grad_s = Q[idx,:] @ alpha - np.ones(idx.shape[0])
        proj_grad_s = grad_s - np.dot(y_s,grad_s)/np.dot(y_s, y_s)*y_s

        bound = np.zeros(idx.shape[0])
        bound[proj_grad_s < 0] = self.C

        eta = np.min(np.abs(alpha_s-bound)/(np.abs(proj_grad_s)+0.001))

        alpha[idx] -= min(eta, self.lr) * proj_grad_s

      L = 0.5 * alpha.T @ Q @ alpha - np.sum(alpha) # objectibve function
      if (L > prev_L):
        if (self.debug>0):
          print(f'Early stopping at epoch={epoch}! (reduce learning rate lr)')
        break

      if (self.debug>1):
        print(f'[PGD optimizer] epoch = {epoch}: L = {L:.5f}  (# of support vectors = {(alpha>self.threshold).sum()})')
        print(f'                 alpha: max={np.max(alpha)} min={np.min(alpha)} orthogonal constraint={np.dot(alpha,y):.2f}')

      prev_L = L

    return alpha

  # train SVM from training samples
  # X[N,d]: input features;  y[N]: output labels (+1 or -1)
  def fit(self, X, y):
    if(self.kernel != 'linear' and self.kernel != 'poly' and self.kernel != 'rbf'):
      print("Error: only linear/poly/rbf kernel is supported!")
      return

    Q = self.QuadraticMatrix(X, y)

    alpha = self.PGD(Q, y)

    #save support vectors (pruning all data with alpha==0)
    self.X_SVs = X[alpha>self.threshold]
    self.y_SVs = y[alpha>self.threshold]
    self.alpha_SVs = alpha[alpha>self.threshold]

    if(self.kernel == 'linear'):
      self.w = (self.y_SVs * self.alpha_SVs) @ self.X_SVs

    # estimate b
    idx = np.nonzero(np.logical_and(self.alpha_SVs>self.threshold,self.alpha_SVs<self.C-self.threshold))
    if(len(idx) == 0):
      idx = np.nonzero(self.alpha_SVs>self.threshold)
    # refer to the formula on page 125 (above Figure 6.11)
    b = self.y_SVs[idx] - (self.y_SVs * self.alpha_SVs) @ self.Kernel(self.X_SVs, self.X_SVs[idx])
    self.b = np.median(b)

    return

  # use SVM from prediction
  # X[N,d]: input features
  def predict(self, X):
    if(self.kernel != 'linear' and self.kernel != 'poly' and self.kernel != 'rbf'):
      print("Error: only linear/poly/rbf kernel is supported!")
      return

    if(self.kernel == 'linear'):
      y = X @ self.w + self.b
    else:
      y = (self.y_SVs * self.alpha_SVs) @ self.Kernel(self.X_SVs, X) + self.b

    return np.sign(y)

Displaying the accuracy with different C and gamma values

In [ ]:
C_values = [0.1, 1, 2, 4, 10]
gamma_values = [0.01, 0.1, 1, 2, 10]

for c in C_values:
  for g in gamma_values:
    svm = mySVM2(max_epochs=20, lr=1.0, C=c, kernel='rbf', gamma=g, debug=0)
    svm.fit(X_train, y_train)
    predict_train = svm.predict(X_train)
    train_acc = np.count_nonzero(np.equal(predict_train, y_train)) / y_train.size
    predict_test = svm.predict(X_test)
    test_acc = np.count_nonzero(np.equal(predict_test, y_test)) / y_test.size
    print(f'MY RBF SVM (C={c}, gamma={g}): training accuracy={100*train_acc:.2f}%  test accuracy={100*test_acc:.2f}%')

MY RBF SVM (C=0.1, gamma=0.01): training accuracy=92.43%  test accuracy=93.23%
MY RBF SVM (C=0.1, gamma=0.1): training accuracy=94.63%  test accuracy=95.04%
MY RBF SVM (C=0.1, gamma=1): training accuracy=97.65%  test accuracy=97.79%
MY RBF SVM (C=0.1, gamma=2): training accuracy=98.38%  test accuracy=98.28%
MY RBF SVM (C=0.1, gamma=10): training accuracy=99.55%  test accuracy=96.66%
MY RBF SVM (C=1, gamma=0.01): training accuracy=94.45%  test accuracy=94.99%
MY RBF SVM (C=1, gamma=0.1): training accuracy=96.37%  test accuracy=96.71%
MY RBF SVM (C=1, gamma=1): training accuracy=99.62%  test accuracy=98.87%
MY RBF SVM (C=1, gamma=2): training accuracy=99.81%  test accuracy=99.36%
MY RBF SVM (C=1, gamma=10): training accuracy=100.00%  test accuracy=98.97%
MY RBF SVM (C=2, gamma=0.01): training accuracy=94.95%  test accuracy=95.48%
MY RBF SVM (C=2, gamma=0.1): training accuracy=96.91%  test accuracy=97.10%
MY RBF SVM (C=2, gamma=1): training accuracy=99.75%  test accuracy=98.97%
MY RBF SVM

Displaying the timing

In [ ]:
c = 1
gamma = 0.1

# Scikit-learn RBF SVM
rbfSVM = SVC(kernel='rbf', C=c, gamma=gamma)
%timeit rbfSVM.fit(X_train, y_train)

# PGD-based RBF SVM
svm = mySVM2(max_epochs=10, lr=2.0, C=c, kernel='rbf', gamma=gamma)
%timeit svm.fit(X_train, y_train)

22 s ± 285 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
18.8 s ± 352 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
